In [1]:
import gdelt
import pandas as pd
import numpy as  np
import seaborn as sns
import math
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,LabelBinarizer,OneHotEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, make_scorer,mean_squared_error,r2_score

Using TensorFlow backend.


In [2]:
gd = gdelt.gdelt()

df=gd.Search(['2017 10 01','2018 02 18'],table='events')
df = df[df.EventRootCode != '--']
df.index = df.SQLDATE
df.sort_index(inplace=True)
df.index = pd.to_datetime(df.index, format='%Y%m%d')
df.drop(['SQLDATE','QuadClass'], axis=1, inplace=True)

/home/chiru/anaconda2/lib/python2.7/site-packages/gdelt/parallel.py:69: UserWarning: GDELT does not have a url for date time 20171001234500
  warnings.warn(message)
/home/chiru/anaconda2/lib/python2.7/site-packages/gdelt/parallel.py:105: UserWarning: GDELT did not return data for date time 20171001234500
  warnings.warn(message)
/home/chiru/anaconda2/lib/python2.7/site-packages/gdelt/parallel.py:69: UserWarning: GDELT does not have a url for date time 20171030234500
  warnings.warn(message)
/home/chiru/anaconda2/lib/python2.7/site-packages/gdelt/parallel.py:105: UserWarning: GDELT did not return data for date time 20171030234500
  warnings.warn(message)
/home/chiru/anaconda2/lib/python2.7/site-packages/gdelt/parallel.py:69: UserWarning: GDELT does not have a url for date time 20171130234500
  warnings.warn(message)
/home/chiru/anaconda2/lib/python2.7/site-packages/gdelt/parallel.py:105: UserWarning: GDELT did not return data for date time 20171130234500
  warnings.warn(message)
/home/c

In [3]:
df.shape

(213439, 60)

In [4]:
def get_data(df,country,code):
    scaler = MinMaxScaler(feature_range=(0,50))
    df.NumMentions = scaler.fit_transform(df['NumMentions'].reshape(-1,1))
    df = df[df.ActionGeo_CountryCode == country]
    indices = df.index.unique()
    aggregated_df = pd.DataFrame(columns=['ActionGeo_CountryCode', 'EventRootCode', 'NumMentions','GoldsteinScale', 'AvgTone'], index= indices)
    aggregated_df.ActionGeo_CountryCode = country
    aggregated_df.EventRootCode = code
    for i in range(len(indices)):
        aggregated_df.NumMentions[indices[i]] = df.NumMentions[indices[i]].sum()
        aggregated_df.GoldsteinScale[indices[i]] = df.GoldsteinScale[indices[i]].max()
        aggregated_df.AvgTone[indices[i]] = df.AvgTone[indices[i]].mean()
    
    return aggregated_df

In [5]:
import numpy as np
import pandas as pd
import scipy.stats as sts



def avg(data):
    return data.sum()/len(data)

def stand_dev(newl):
    npstd=np.std(newl,axis=None)
    return npstd

def Kurtosis(newl):
    krt=sts.kurtosis(newl,axis=None)
    return krt

#skewness
def skewness(newl):
    skw=sts.skew(newl,axis=None)
    return skw


def getIndicators(aggregated_df): 
    df = aggregated_df
    
    lag_day = 7
    technical_indicator = pd.DataFrame(columns=['avgNumMentions', 'avgAvgTone', 'stand_dev', 'Kurtosis',
                                                'skewness'],index = [i for i in range(df.shape[0]-lag_day)])
    for i in range(df.shape[0]-lag_day):
        technical_indicator.iloc[i].avgNumMentions = avg(df.NumMentions[i:i+10])
        technical_indicator.iloc[i].avgAvgTone = avg(df.AvgTone[i:i+10])
        technical_indicator.iloc[i].stand_dev = stand_dev(df.NumMentions[i:i+10])
        technical_indicator.iloc[i].Kurtosis = Kurtosis(df.NumMentions[i:i+10])
        technical_indicator.iloc[i].skewness = skewness(df.NumMentions[i:i+10])
        
    return technical_indicator

In [6]:
def create_data(df,lag_days):
    dataX, dataY = [], []
    for i in range(len(df)-lag_days-1):
        a = df[i:(i+lag_days)]
        dataX.append(np.array(a))
        dataY.append(df.iloc[i+lag_days].avgNumMentions)
    
    return np.array(dataX), np.array(dataY)    

In [7]:
#LSTM for cluster 1
def indicator_lstm(technicalIndicators, plot=True):
    df = technicalIndicators
    total_days = len(df)
    
    train_len = int(0.75*total_days)
    train = df[:train_len]
    test = df[train_len:]
    
    X_train, y_train = create_data(train, 6)
    X_test, y_test = create_data(test, 6)
    
    batch_size = 4
    epochs = 50
    
    model = Sequential()
    model.add(LSTM(512, batch_input_shape=(None, 6, X_train.shape[2])))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss= 'logcosh', optimizer= 'Adagrad')
    history = model.fit(X_train,y_train, epochs=epochs, batch_size=batch_size, verbose=1, validation_data=(X_test, y_test))
    
    
    trainPredict = model.predict(X_train)
    testPredict = model.predict(X_test)
    
    if plot==True:
        plt.figure(figsize=(12,7))
        plt.plot(history.history['loss'], c='r', label='train_loss')
        plt.plot(history.history['val_loss'], c='g', label='validation_loss')
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.title('Epoch vs Loss curve')
        plt.legend(loc='best')
        
        
        
        plt.figure(figsize=(12,7))
        plt.plot(testPredict,  c='r', label="testPredict")
        plt.plot(y_test, label="test")
        plt.title('Actual vs Predicted test set')
        plt.legend(loc='best')
        
        plt.figure(figsize=(12,7))
        plt.plot(trainPredict, c='r',label="trainPredict")
        plt.plot(y_train, label="train")
        plt.title('Actual vs Predicted on training set')
        plt.legend(loc='best')
        
        train_NumMentions = df.avgNumMentions
    
        X_actual_data, y_actual_data = create_data(df, 6)
        predicted_whole_series = model.predict(X_actual_data)
        
        plt.figure(figsize=(12,7))
        plt.plot(predicted_whole_series, c='g', label="Predicted whole series")
        plt.plot(train_NumMentions, c='r', label="Training series")
        plt.title('Actual training vs whole series as prediction')
        plt.legend(loc='best')
    
    y_test = y_test.reshape(y_test.shape[0],1)
    y_train = y_train.reshape(y_train.shape[0],1)

    print("r-sq(train): ",r2_score(y_train, model.predict(X_train)))
    print("r-sq(test): ",r2_score(y_test, testPredict))
    print("rmse(train) :", mean_squared_error(y_train, model.predict(X_train)))
    print("rmse(test) :", mean_squared_error(y_test, testPredict))
    print()
    print()

    return testPredict

In [8]:
country='IN'
code=14
data=get_data(df,country,code)

/home/chiru/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  This is separate from the ipykernel package so we can avoid doing imports until
/home/chiru/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/home/chiru/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/home/chiru/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [9]:
indicator_lstm(getIndicators(data), plot=True)

Train on 107 samples, validate on 31 samples
Epoch 1/50
107/107 [==============================] - 3s 26ms/step - loss: 5.1240 - val_loss: 6.5764
Epoch 2/50
107/107 [==============================] - 2s 16ms/step - loss: 3.2487 - val_loss: 6.1302
Epoch 3/50
107/107 [==============================] - 2s 16ms/step - loss: 2.9584 - val_loss: 6.2724
Epoch 4/50
107/107 [==============================] - 2s 17ms/step - loss: 2.8407 - val_loss: 5.7616
Epoch 5/50
107/107 [==============================] - 2s 17ms/step - loss: 2.7139 - val_loss: 5.6227
Epoch 6/50
107/107 [==============================] - 2s 17ms/step - loss: 2.8332 - val_loss: 5.2985
Epoch 7/50
107/107 [==============================] - 2s 17ms/step - loss: 2.5791 - val_loss: 4.9918
Epoch 8/50
107/107 [==============================] - 2s 17ms/step - loss: 2.5533 - val_loss: 5.2767
Epoch 9/50
107/107 [==============================] - 2s 17ms/step - loss: 2.6066 - val_loss: 4.7952
Epoch 10/50
107/107 [=========================

array([[ 36.37938309],
       [ 35.97980118],
       [ 36.13120651],
       [ 35.71194458],
       [ 36.24080658],
       [ 36.89168549],
       [ 36.37873077],
       [ 34.06826401],
       [ 33.61024475],
       [ 33.43118286],
       [ 33.69174576],
       [ 33.50528717],
       [ 33.23389435],
       [ 32.37353897],
       [ 30.66086769],
       [ 25.05683136],
       [ 23.09456253],
       [ 23.69203949],
       [ 24.45763779],
       [ 24.56210709],
       [ 22.96327782],
       [ 24.9401741 ],
       [ 23.62578964],
       [ 24.6068058 ],
       [ 24.88710785],
       [ 25.30991364],
       [ 25.57856369],
       [ 27.60268402],
       [ 30.32599449],
       [ 33.09531403],
       [ 34.19811249]], dtype=float32)